huggingface의 monologg/koelectra-base-v3-discriminator 모델을 사용하여 문제의 클래스를 예측하는 과정이 담긴 코드입니다.

#라이브러리 설치

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 4.2 MB/s 
     |████████████████████████████████| 86 kB 6.2 MB/s 
     |████████████████████████████████| 596 kB 66.2 MB/s 
     |████████████████████████████████| 6.6 MB 53.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


# 1- 데이터 불러오기

In [ ]:
import pandas as pd
import numpy as np

data_path = '/content/drive/MyDrive/KMWP/code/data/new_train.csv'

data = pd.read_csv(data_path)
data.head()

,index,class,problem,code,answer,Unnamed: 5
0,0,8,한 변의 길이가 24cm인 정육각형과 둘레가 같은 정팔각형이 있습니다. 이 정팔각형...,a = 24\nb = 6\nc = 8\ny = a * b // c\nprint(y),18,NaN
1,1,1,윤아는 부추전을 똑같이 8조각으로 나누어 한 조각을 먹었습니다. 윤미는 같은 크기의...,a = 8\nb = 16\ny = b // a\nprint(y),2,NaN
2,2,8,"화단 주변에 한 변이 12m인 정팔각형 모양의 울타리를 두른다면, 울타리는 모두 몇...",a = 12\nb = 8\ny = a * b\nprint(y),96,NaN
3,3,3,"6장의 숫자 카드 0, 9, 8, 7, 2, 1가 있습니다. 이를, 한 번씩 사용하...",from itertools import permutations\n\na = ['0'...,987210,NaN
4,4,1,0.26 x 0.8을 계산해 주세요.,"a = 0.26\nb = 0.8\ny = a * b\nprint(""{:.2f}"".f...",0.21,NaN


In [ ]:
data.drop(columns=['index', 'Unnamed: 5'], axis=1, inplace=True) 
data.head()

,class,problem,code,answer
0,8,한 변의 길이가 24cm인 정육각형과 둘레가 같은 정팔각형이 있습니다. 이 정팔각형...,a = 24\nb = 6\nc = 8\ny = a * b // c\nprint(y),18
1,1,윤아는 부추전을 똑같이 8조각으로 나누어 한 조각을 먹었습니다. 윤미는 같은 크기의...,a = 8\nb = 16\ny = b // a\nprint(y),2
2,8,"화단 주변에 한 변이 12m인 정팔각형 모양의 울타리를 두른다면, 울타리는 모두 몇...",a = 12\nb = 8\ny = a * b\nprint(y),96
3,3,"6장의 숫자 카드 0, 9, 8, 7, 2, 1가 있습니다. 이를, 한 번씩 사용하...",from itertools import permutations\n\na = ['0'...,987210
4,1,0.26 x 0.8을 계산해 주세요.,"a = 0.26\nb = 0.8\ny = a * b\nprint(""{:.2f}"".f...",0.21


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2820 entries, 0 to 2819
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   class    2820 non-null   int64 
 1   problem  2820 non-null   object
 2   code     2820 non-null   object
 3   answer   2820 non-null   object
dtypes: int64(1), object(3)
memory usage: 88.2+ KB


In [ ]:
data = data.astype({'class':'int32'})
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2820 entries, 0 to 2819
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   class    2820 non-null   int32 
 1   problem  2820 non-null   object
 2   code     2820 non-null   object
 3   answer   2820 non-null   object
dtypes: int32(1), object(3)
memory usage: 77.2+ KB


주어진 test 데이터에는 class가 없기 때문에 정확도를 뽑기 위해 train 데이터를 분리한다.

In [ ]:
data_length = int(len(data) * 0.9)

data_length

2538

In [ ]:
test_data = data[data_length:]
data = data[:data_length] 


len(data), len(test_data)

(2538, 282)

숫자형태의 카테고리로 인코딩을 수행한다.   
-> 그래야 loss가 잘 계산됨

In [ ]:
# label encoding 
from sklearn.preprocessing import LabelEncoder 

label_encoder = LabelEncoder() 
label_encoder.fit(data['class'])
num_labels = len(label_encoder.classes_) 

data['encoded_label'] = np.asarray(label_encoder.transform(data['class']), dtype=np.int32)
data.head() 

,class,problem,code,answer,encoded_label
0,8,한 변의 길이가 24cm인 정육각형과 둘레가 같은 정팔각형이 있습니다. 이 정팔각형...,a = 24\nb = 6\nc = 8\ny = a * b // c\nprint(y),18,7
1,1,윤아는 부추전을 똑같이 8조각으로 나누어 한 조각을 먹었습니다. 윤미는 같은 크기의...,a = 8\nb = 16\ny = b // a\nprint(y),2,0
2,8,"화단 주변에 한 변이 12m인 정팔각형 모양의 울타리를 두른다면, 울타리는 모두 몇...",a = 12\nb = 8\ny = a * b\nprint(y),96,7
3,3,"6장의 숫자 카드 0, 9, 8, 7, 2, 1가 있습니다. 이를, 한 번씩 사용하...",from itertools import permutations\n\na = ['0'...,987210,2
4,1,0.26 x 0.8을 계산해 주세요.,"a = 0.26\nb = 0.8\ny = a * b\nprint(""{:.2f}"".f...",0.21,0


In [ ]:
problem = data.problem.to_list()
classes = data['encoded_label'].to_list() 

In [ ]:
problem[0], classes[:2]

('한 변의 길이가 24cm인 정육각형과 둘레가 같은 정팔각형이 있습니다. 이 정팔각형의 한 변의 길이는 몇 cm인지 구하시오.',
 [7, 0])

텍스트와 라벨을 따로 분리한다.

In [ ]:
from sklearn.model_selection import train_test_split 

train_prob, val_prob, train_class, val_class = train_test_split(problem,
                                                                classes, 
                                                                test_size=0.1, 
                                                                random_state=5)

In [ ]:
len(train_prob)

2284

# 2- 모델 및 토크나이저 불러오기
hugging face에서 monologg/koelectra-base-v3-discriminator 모델과   
Electra 토크나이저를 가져온다.

In [ ]:
from transformers import ElectraTokenizer

model_path = 'monologg/koelectra-base-v3-discriminator'

tokenizer = ElectraTokenizer.from_pretrained(model_path)

# 문제 토큰화
train_encodings = tokenizer(train_prob, truncation=True, padding=True)
val_encodings = tokenizer(val_prob, truncation=True, padding=True) 

Downloading:   0%|          | 0.00/257k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/61.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/467 [00:00<?, ?B/s]

토큰화 된 데이터셋을 Tensorflow의 Dataset object로 변환

In [ ]:
import tensorflow as tf  

# train set
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_class
))

# validation set
val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings), 
    val_class
))

In [ ]:
train_dataset

<TensorSliceDataset element_spec=({'input_ids': TensorSpec(shape=(117,), dtype=tf.int32, name=None), 'token_type_ids': TensorSpec(shape=(117,), dtype=tf.int32, name=None), 'attention_mask': TensorSpec(shape=(117,), dtype=tf.int32, name=None)}, TensorSpec(shape=(), dtype=tf.int32, name=None))>

# 3- Koelectra fine-tuning 하기
Text Classification 이 목적이므로 TFElectraForSequenceClassification 클래스를 활용한다.

In [ ]:
from transformers import TFElectraForSequenceClassification

num_labels = len(label_encoder.classes_)

model = TFElectraForSequenceClassification.from_pretrained(model_path,
                                                           num_labels = num_labels,
                                                           from_pt=True)

optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5) 
model.compile(optimizer = optimizer, )#loss=model.compute_loss, metrics=['accuracy'],)

Downloading:   0%|          | 0.00/431M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.weight', 'electra.embeddings.position_ids', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing TFElectraForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFElectraForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFElectraForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.dens

In [ ]:
#!pip install pyyaml h5py

모델 학습시키기

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import os 

callback_es = EarlyStopping(
    monitor = 'val_loss',
    patience = 2
)

mc = ModelCheckpoint('training_1/cp.ckpt', monitor='val_loss', mode='min', verbose = 1,
                     save_best_only=True)


model.fit(
    train_dataset.shuffle(2538).batch(16), epochs = 10, batch_size = 16,
    validation_data = val_dataset.shuffle(1000).batch(16),
    callbacks = [callback_es, mc]
)


Epoch 1/10
143/143 [==============================] - ETA: 0s - loss: 1.0162
Epoch 1: val_loss improved from inf to 0.52088, saving model to training_1/cp.ckpt


INFO:tensorflow:Assets written to: training_1/cp.ckpt/assets


INFO:tensorflow:Assets written to: training_1/cp.ckpt/assets


143/143 [==============================] - 96s 524ms/step - loss: 1.0162 - val_loss: 0.5209
Epoch 2/10
143/143 [==============================] - ETA: 0s - loss: 0.4230
Epoch 2: val_loss improved from 0.52088 to 0.40478, saving model to training_1/cp.ckpt


INFO:tensorflow:Assets written to: training_1/cp.ckpt/assets


...

INFO:tensorflow:Assets written to: training_1/cp.ckpt/assets


143/143 [==============================] - 69s 481ms/step - loss: 0.1689 - val_loss: 0.2769
Epoch 7/10
143/143 [==============================] - ETA: 0s - loss: 0.1222
Epoch 7: val_loss improved from 0.27692 to 0.22223, saving model to training_1/cp.ckpt


INFO:tensorflow:Assets written to: training_1/cp.ckpt/assets


INFO:tensorflow:Assets written to: training_1/cp.ckpt/assets


143/143 [==============================] - 68s 480ms/step - loss: 0.1222 - val_loss: 0.2222
Epoch 8/10
143/143 [==============================] - ETA: 0s - loss: 0.0999
Epoch 8: val_loss did not improve from 0.22223
143/143 [==============================] - 37s 258ms/step - loss: 0.0999 - val_loss: 0.2758
Epoch 9/10
143/143 [==============================] - ETA: 0s - loss: 0.1041
Epoch 9: val_loss did not improve from 0.22223
143/143 [==============================] - 37s 258ms/step - loss: 0.1041 - val_loss: 0.3335


In [ ]:
#dir(model)

In [ ]:
train_dataset.element_spec[0]['input_ids']

TensorSpec(shape=(117,), dtype=tf.int32, name=None)

원래는 아래의 코드에서 주석 처리된 부분을 실행해서, 인코딩 된 라벨들을 원상복귀시켜야 하는데,   
어찌된 일인지 계속 에러가 나서 실행하지 않았다.  

이 부분은 아래에서 다시 처리할 예정이다.

In [ ]:
import re

id2labels = model.config.id2label
#print(id2labels)
#model.config.id2label = {id: label_encoder.inverse_transform([int(re.sub('LABEL_', '', label))])[0] for id, label in id2labels.items()}
print(id2labels)

label2ids = model.config.label2id  
#model.config.label2id = {label_encoder.inverse_transform([int(re.sub('LABEL_', '', label))])[0] :id for id, label in id2labels.items()}
print(label2ids)

{0: 'LABEL_0', 1: 'LABEL_1', 2: 'LABEL_2', 3: 'LABEL_3', 4: 'LABEL_4', 5: 'LABEL_5', 6: 'LABEL_6', 7: 'LABEL_7'}
{'LABEL_0': 0, 'LABEL_1': 1, 'LABEL_2': 2, 'LABEL_3': 3, 'LABEL_4': 4, 'LABEL_5': 5, 'LABEL_6': 6, 'LABEL_7': 7}


모델 저장하기

In [ ]:
model.save_pretrained('finetuned_koelectra.h5')
tokenizer.save_pretrained('finetuned_koelectra')

('finetuned_koelectra/tokenizer_config.json',
 'finetuned_koelectra/special_tokens_map.json',
 'finetuned_koelectra/vocab.txt',
 'finetuned_koelectra/added_tokens.json')

# 4- 저장된 모델 불러와서 클래스 예측하기

In [ ]:
# 위에서 분리한 test_data를 가져온다
test_data.head() 

,class,problem,code,answer
2538,7,"상현이는 물을 2리터의 0.5 배만큼 마셨고 승기는 0.8 리터 를 마셨다면, 물을...","a = 2\nb = 0.5\nc = 0.8\ndicts = {'상현': a * b,...",상현
2539,8,한 변의 길이가 12cm인 평행사변형과 한 변의 길이가 24cm인 정팔각형의 둘레가...,a = 12\nb = 24\nc = 8\ny = (b * c - a * 2) // ...,84
2540,1,서진이네 반 학생들을 한조에 4명씩 조를 나누려고 합니다. 학생이 36명이라면 몇 ...,a = 4\nb = 36\ny = b // a\nprint(y),9
2541,1,1시간에 3분씩 빨라지는 시계가 있습니다. 이 시계의 시각을 오늘 오후 4시에 정확...,a = 2\nb = 4\nc = 11\ny = (c - b) * a\nprint(y),14
2542,8,가로가 9센티미터이고 세로는 가로보다 11센티미터 더 긴 직사각형의 둘레는 몇 센티...,a = 9\nb = 11\ny = (a + a + b) * 2\nprint(y),58


In [ ]:
# 위에서 저장한 모델 및 토크나이저 가져오기
from transformers import TextClassificationPipeline 

loaded_tokenizer = ElectraTokenizer.from_pretrained('finetuned_koelectra')
loaded_model = TFElectraForSequenceClassification.from_pretrained('finetuned_koelectra.h5')

text_classifier = TextClassificationPipeline(
    tokenizer = loaded_tokenizer,
    model=loaded_model,
    framework='tf',
    return_all_scores=True
)


All model checkpoint layers were used when initializing TFElectraForSequenceClassification.

All the layers of TFElectraForSequenceClassification were initialized from the model checkpoint at finetuned_koelectra.h5.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFElectraForSequenceClassification for predictions without further training.


from_pretrained() 메소드에 저장된 모델 디렉토리를 넣어 fine-tuning한 model 및 tokenizer를 가져올 수 있다.  

transformers의 Pipelines 클래스를 사용하면 특정 task에 대한 inference를 수행할 수 있다.

In [ ]:
# 클래스 예측하기
predicted_label_list = [] 
predicted_score_list = [] 

for text in test_data['problem']: 
  #predict 
  preds_list = text_classifier(text)[0]

  sorted_preds_list = sorted(preds_list, key=lambda x: x['score'], reverse=True)
  
  predicted_label_list.append(sorted_preds_list[0]['label']) 
  predicted_score_list.append(sorted_preds_list[0]['score'])

결과가 어떻게 나오는지 확인

In [ ]:
preds_list

[{'label': 'LABEL_0', 'score': 0.9990197420120239},
 {'label': 'LABEL_1', 'score': 1.8414139049127698e-05},
 {'label': 'LABEL_2', 'score': 7.963629286678042e-06},
 {'label': 'LABEL_3', 'score': 5.761999636888504e-05},
 {'label': 'LABEL_4', 'score': 0.0005127033800818026},
 {'label': 'LABEL_5', 'score': 3.082417606492527e-05},
 {'label': 'LABEL_6', 'score': 8.989063644548878e-05},
 {'label': 'LABEL_7', 'score': 0.000262811838183552}]

In [ ]:
preds_list[0].items()

dict_items([('label', 'LABEL_0'), ('score', 0.9990197420120239)])

In [ ]:
len(predicted_label_list)
predicted_label_list[:5]

['LABEL_6', 'LABEL_7', 'LABEL_0', 'LABEL_0', 'LABEL_7']

In [ ]:
predicted_score_list[:5]

[0.9938551783561707,
 0.9990087747573853,
 0.9989891648292542,
 0.9963825941085815,
 0.999076247215271]

위에서 인코딩한 라벨을 처리해준다.   
LABEL_을 빼고 1을 더해 원래 class 값으로 저장한다.

In [ ]:
predicted_label_list_2 = []
for i in range(len(predicted_label_list)):
  predicted_label_list_2.append(re.sub('LABEL_', '', predicted_label_list[i]))
  predicted_label_list_2= list(map(int, predicted_label_list_2))
  predicted_label_list_2[i] += 1
predicted_label_list_2[:5]

[7, 8, 1, 1, 8]

In [ ]:
len(sorted_preds_list)

8

test_data에 예측한 클래스와 확률값을 추가하여 확인한다.

In [ ]:
test_data['pred'] = predicted_label_list_2 
test_data['score'] = predicted_score_list
test_data.head()

,class,problem,code,answer,pred,score
2538,7,"상현이는 물을 2리터의 0.5 배만큼 마셨고 승기는 0.8 리터 를 마셨다면, 물을...","a = 2\nb = 0.5\nc = 0.8\ndicts = {'상현': a * b,...",상현,7,0.993855
2539,8,한 변의 길이가 12cm인 평행사변형과 한 변의 길이가 24cm인 정팔각형의 둘레가...,a = 12\nb = 24\nc = 8\ny = (b * c - a * 2) // ...,84,8,0.999009
2540,1,서진이네 반 학생들을 한조에 4명씩 조를 나누려고 합니다. 학생이 36명이라면 몇 ...,a = 4\nb = 36\ny = b // a\nprint(y),9,1,0.998989
2541,1,1시간에 3분씩 빨라지는 시계가 있습니다. 이 시계의 시각을 오늘 오후 4시에 정확...,a = 2\nb = 4\nc = 11\ny = (c - b) * a\nprint(y),14,1,0.996383
2542,8,가로가 9센티미터이고 세로는 가로보다 11센티미터 더 긴 직사각형의 둘레는 몇 센티...,a = 9\nb = 11\ny = (a + a + b) * 2\nprint(y),58,8,0.999076


# 5- 평가하기

In [ ]:
from sklearn.metrics import classification_report 

print(classification_report(y_true=test_data['class'], y_pred=test_data['pred']))

              precision    recall  f1-score   support

           1       0.95      0.98      0.97        99
           2       1.00      0.67      0.80         3
           3       0.94      0.79      0.86        19
           4       0.78      0.75      0.77        24
           5       0.84      0.89      0.86        18
           6       0.90      0.86      0.88        21
           7       0.92      0.92      0.92        26
           8       0.97      1.00      0.99        72

    accuracy                           0.93       282
   macro avg       0.91      0.86      0.88       282
weighted avg       0.93      0.93      0.93       282



- 데이터 불균형을 해결하지 않은 데이터로 계산한 결과 93%의 정확도를 보인다. 
